In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/Users/sairam/Desktop/attention_is_all_you_need.pdf")
docs = loader.load()
print(type(docs))

<class 'list'>


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
list_documents=text_splitter.split_documents(docs)


In [3]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [4]:
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
google_api_key=os.getenv('GOOGLE_API_KEY')

In [5]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=google_api_key)
vectorstore = Chroma.from_documents(docs, embeddings)
query='what is this paper all about'
docs = vectorstore.similarity_search(query, k=3)


/opt/anaconda3/envs/django/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1722423524.716495  739736 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [6]:
docs

[Document(metadata={'page': 0, 'source': '/Users/sairam/Desktop/attention_is_all_you_need.pdf'}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks sho

In [18]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import StuffDocumentsChain

llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=google_api_key)


In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [10]:
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x10f8d9700>)

In [11]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [12]:
response=retrieval_chain.invoke({"input":query})

I0000 00:00:1722423528.631298  739884 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [13]:
response['answer']

'This paper is about a new neural network architecture called Transformer, which is based entirely on attention mechanisms.\nTransformer is superior to previous models in terms of quality, parallelizability and training time.'

In [ ]:
aws ecr create-repository --repository-name first_ecr_repo
